# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Linking microscopical stochastic gene expression dynamics with macroecological patterns in phytoplankton communities - a theoretical approach	Contributed Talk	talk-6	Random Dynamical Systems,  with applications in biology	2024-11-06	Chicago, U.S.A.	https://www.nitmb.org/random-dynamical-systems	National Institute for Theory and Mathematics in Biology (NITMB) Workshop
Delay effects on the stability of large ecosystems	Invited Talk	talk-5	Italian Society of Statistical Physics - Young seminars	2024-06-13	Padova, Italy	https://www.youtube.com/watch?v=O-VFbhJYoJs	Invited Talk at the Italian Society of Statistical Physics - Young seminars
Complexity and emergence in marine ecosystems/seascape: theory, mechanisms and data analysis	Contributed Talk	talk-4	Synergy Summer School 2024	2024-06-26	Ischia, Italy	https://www.atlanteco.eu/synergy-summer-school-2024	AtlantECO, Mission Atlantic and BIOcean5D Summer School
Deviation from neutr

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Linking microscopical stochastic gene expressi...,Contributed Talk,talk-6,"Random Dynamical Systems, with applications i...",2024-11-06,"Chicago, U.S.A.",https://www.nitmb.org/random-dynamical-systems,National Institute for Theory and Mathematics ...
1,Delay effects on the stability of large ecosys...,Invited Talk,talk-5,Italian Society of Statistical Physics - Young...,2024-06-13,"Padova, Italy",https://www.youtube.com/watch?v=O-VFbhJYoJs,Invited Talk at the Italian Society of Statist...
2,Complexity and emergence in marine ecosystems/...,Contributed Talk,talk-4,Synergy Summer School 2024,2024-06-26,"Ischia, Italy",https://www.atlanteco.eu/synergy-summer-school...,"AtlantECO, Mission Atlantic and BIOcean5D Summ..."
3,Deviation from neutral species abundance distr...,Contributed Talk,talk-3,ICTP,2024-06-04,"Trieste, Italy",http://pols_italy.unimi.it/pols2024/,Annual meeting of the Physics of Living System...
4,Deviation from neutral species abundance distr...,Contributed Talk,talk-3b,REIMAGINING OCEAN ECOLOGY - Towards a Paradigm...,2024-03-13,"Venice, Italy",https://liphlab.github.io/REIMOCEAN24/,Co-organizer of the workshop
5,Unveiling Global Patterns in Taxonomic and Gen...,Contributed Talk,talk-2,ICTP,2023-10-10,"Trieste, Italy",https://indico.ictp.it/event/10213,Advanced School on Quantitative Principles in ...
6,Modeling bio-geographies – a conceptual approach,Contributed Talk,talk-1,AtlantECO and CEODOS,2023-09-25,"Santiago, Chile",https://eventos.cmm.uchile.cl/omicsceodos/,CEODOS and AtlantECO Workshop


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [7]:
!ls ../_talks

2023-09-25-talk-1.md  2024-06-04-talk-3.md  2024-11-06-talk-6.md
2023-10-10-talk-2.md  2024-06-13-talk-5.md
2024-03-13-talk-3b.md 2024-06-26-talk-4.md


In [8]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
